In [1]:
%load_ext autoreload
%autoreload 2

import os
from tempfile import TemporaryDirectory
from loguru import logger

import pandas as pd
import numpy as np
from scipy.stats import spearmanr
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold

import plotnine as p9

from predictability.models import ResidueAgnosticRegressor, PartialLeastSquares, RITARegressor, PottsRegressor
from predictability.utils import update_environment_variables, read_fasta
from predictability.sm_utils import get_rita_embeddings, get_potts_emissions
from predictability.constants import DATA_ROOT, PROJECT_ROOT, BINARY_RESIDUE_FEATURES
from predictability.plotting import color_mapping, assign_pretty_feature_names, feature_mapping

In [2]:
update_environment_variables("zsh")
DEFAULT_BUCKET = os.environ.get("AWS_DEFAULT_BUCKET")
GLOBAL_DATA_ROOT = os.environ.get("DATA_ROOT")
SEED = 1337
ds = "AMY_BACSU"
target = "stain_activity"
n_folds = 10
model_names = ["Residue agnostic regressor", "Partial least squares", "Augmented Potts", "RITA regressor"]
lambda_reg = 0.1
results_dir = PROJECT_ROOT / f"results/{ds}/combinatorials"
results_dir.mkdir(exist_ok=True, parents=True)
reference = [sequence for _, sequence in read_fasta(DATA_ROOT / "AMY_BACSU/reference.fasta").items()][0]

In [3]:
rita_training_job_names = {
    "AMY_BACSU": "huggingface-regressor-01-08-09-05"
}
potts_training_job_names = {
    "AMY_BACSU": "potts-regressor-12-09-53-35"
}

In [4]:
results = {
    "dataset": [],
    "model": [],
    "fold": [],
    "structural_characteristic": [],
    "is_member": [],
    "spearman": [],
    "mae": []
}

data = pd.read_csv(DATA_ROOT / f"AMY_BACSU/combinatorials.csv").reset_index()
data["embedding_index"] = list(range(len(data)))
data = data.dropna(subset=target)
embedding_indices = data["embedding_index"].values

with TemporaryDirectory() as temp_dir:
    potts_path = get_potts_emissions(
        path=temp_dir,
        dataset=ds,
        training_job_names=potts_training_job_names,
        default_bucket=DEFAULT_BUCKET
    )
    embeddings_path = get_rita_embeddings(
        path=temp_dir,
        dataset=ds,
        training_job_names=rita_training_job_names,
        default_bucket=DEFAULT_BUCKET
    )
    embeddings = np.load(embeddings_path)
    models = [
        ResidueAgnosticRegressor(wildtype_sequence=reference),
        PartialLeastSquares(),
        PottsRegressor(potts_path=potts_path, alpha=lambda_reg),
        RITARegressor(alpha=lambda_reg)
    ]

    for feature in BINARY_RESIDUE_FEATURES:
        logger.info(f"Feature: {feature}.")
        for is_member in [0, 1]:
            df = data[data[feature]==is_member].reset_index(drop=True)
            logger.info(f"Is member: {['no', 'yes'][is_member]}.")

            k_fold = KFold(n_splits=10, shuffle=True, random_state=SEED)
            for i, (train_indices, test_indices) in enumerate(k_fold.split(np.arange(len(df)))):
                logger.info(f"Starting fold {i}.")
                split_ids = np.empty(len(df), dtype=object)
                split_ids[train_indices] = "train"
                split_ids[test_indices] = "test"
                df["split"] = split_ids
                train_embeddings_indices = df[df["split"] == "train"]["embedding_index"].values
                test_embeddings_indices = df[df["split"] == "test"]["embedding_index"].values
                
                train_data = df[df["split"] == "train"]
                test_data = df[df["split"] == "test"]

                for model, model_name in zip(models, model_names):
                    logger.info(f"Fitting {model_name}.")
                    if model_name == "RITA regressor":
                        model.fit(train_data, target=target, embeddings=embeddings[train_embeddings_indices])
                        test_predictions = model.predict(test_data, embeddings=embeddings[test_embeddings_indices])
                    else:
                        model.fit(train_data, target=target)
                        test_predictions = model.predict(test_data)
                    mae = mean_absolute_error(test_data[target].values, test_predictions)
                    spearman, _ = spearmanr(test_data[target].values, test_predictions)

                    results["dataset"].append(ds)
                    results["model"].append(model_name)
                    results["fold"].append(i)
                    results["structural_characteristic"].append(feature)
                    results["is_member"].append(is_member)
                    results["mae"].append(mae)
                    results["spearman"].append(spearman)

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/floris/Library/Application Support/sagemaker/config.yaml


2024-05-03 16:22:28.843 | INFO     | predictability.models:__init__:149 - Loading Potts model locally from: /var/folders/d8/bxgj52rj3w934zctbjr8v2hh0000gn/T/tmpnnsoxrz3
2024-05-03 16:22:37.195 | INFO     | __main__:<module>:38 - Feature: is_buried.
2024-05-03 16:22:37.216 | INFO     | __main__:<module>:41 - Is member: no.
2024-05-03 16:22:37.220 | INFO     | __main__:<module>:45 - Starting fold 0.
2024-05-03 16:22:37.226 | INFO     | __main__:<module>:57 - Fitting Residue agnostic regressor.
2024-05-03 16:22:37.435 | INFO     | __main__:<module>:57 - Fitting Partial least squares.
2024-05-03 16:22:39.434 | INFO     | __main__:<module>:57 - Fitting Augmented Potts.
2024-05-03 16:22:46.279 | INFO     | __main__:<module>:57 - Fitting RITA regressor.
2024-05-03 16:22:46.317 | INFO     | predictability.models:fit:253 - Fitting ridge
2024-05-03 16:22:46.717 | INFO     | __main__:<module>:45 - Starting fold 1.
2024-05-03 16:22:46.725 | INFO     | __main__:<module>:57 - Fitting Residue agnosti

In [5]:
results_df = pd.DataFrame(results)
results_df.to_csv(results_dir / "results.csv")

In [6]:
results_df["is_member"] = results_df["is_member"].astype(bool)
results_df["Feature label"] = results_df["is_member"].apply(lambda x: "Negative" if x==False else "Positive")
results_df["Feature"] = results_df["structural_characteristic"].apply(assign_pretty_feature_names)

results_df["Structural characteristic"] = results_df[['Feature', 'Feature label']].apply(
    lambda x: feature_mapping[tuple(x)], axis=1
)
results_df["Structural characteristic"] = pd.Categorical(results_df["Structural characteristic"],
                                                     categories=["Buried", "Exposed", "Many contacts", "Few contacts",
                                                                 "Close to active site", "Distant to active site",
                                                                 "Part of helix or sheet", "Part of loop"],
                                                     ordered=True)

p = (
    p9.ggplot(
        results_df,
        p9.aes(
            "Structural characteristic",
            "spearman",
            fill="Structural characteristic",
        )
    )
    + p9.geom_boxplot(show_legend=False)
    + p9.facet_wrap("model", nrow=2, ncol=2)
    + p9.theme(figure_size=(10, 10), axis_text_x=p9.element_text(rotation=15, ha="right"))
    + p9.scale_fill_manual(values=color_mapping)
    + p9.theme(
        plot_background=p9.element_rect(fill='white'),
        panel_background=p9.element_rect(fill='white'),
        axis_title_y=p9.element_text(text="Test Spearman (ρ)", size=18),
    )
    + p9.labs(x="")
)
p.draw()
# p.save(save_dir / "supervised_performance_difference.png")
print(p)